# Artigo utilizado como base para a solução
https://research.ijcaonline.org/volume58/number17/pxc3883886.pdf

# Requisitos da atividade
1) implementar o gerador dos labirintos;

2) implementar o algoritmo genético (AG) com estruturas de dados e parâmetros necessários;

3) executar o AG num labirinto 10x10;

4) traçar um gráfico da evolução do AG (gerações versus fitness da população) usando a biblioteca matplotlib;

5) desenhar uma figura do labirinto e da melhor solução encontrada (pode ser em modo texto);

6) (Bonus) Variar os parâmetros do AG e comentar o efeito sobre a velocidade de convergência da solução.

### Grupo
* Alexandre Ribeiro
* Flávio Farias
* Henrique Arriel

# Imports e constantes

In [ ]:
import random
from random import randint, choice
import matplotlib.pyplot as plt
import numpy as np
from IPython.display import clear_output
from time import sleep

SEED = 1123581321 #seed utilizada para a randomização ser igual em todas execuções do notebook. A modificação da seed pode trazer efeitos colaterais para algumas análises.
rep = { "cross": '#', 'vwall': '#', 'hwall': '#', 'step': '\033[95m+\033[00m', 'start': '\033[92mS\033[00m', 'finish': '\033[91mF\033[00m' }

# Gerador de labirintos (A-Mazer)

# Algoritmo genético

# Execução do algoritmo genético em um labirinto 10x10

# Gráfico com evolução do algoritmo genético (gerações x fitness)

# Figura do labirinto e melhor solução encontrada

# Variação dos parâmetros do algoritmo genético

Para efeitos de comparação das variações de parâmetros do algoritmo genético, reiniciaremos a seed utilzada para gerarmos os mesmos labirintos para os diferentes parâmetros.

Parâmetros base

---
Taxa de crossover == **0.8**

Taxa de mutação == **0.1**

Tamanho da população == **100**

Geração máxima para parada == **500**

Esses parâmetros foram obtidos através do artigo e foram utilizados nas seções anteriores.

Para termos uma média de gerações de convergência do AG, rodamos o algoritmo randômicamente por 50 vezes. 

In [ ]:
random.seed(SEED)

In [ ]:
generation_list = []
for i in range(50):
  _, generation, _, _, _, _ = run(maze_size=10, population_size=100, crossover_rate=0.8, mutation_rate= 0.1, max_generation=500,n_generation_growth= 5)
  generation_list.append(generation)

print("Média de gerações para convergência do Algoritmo Genético: ", np.mean(np.array(generation_list)))

Parâmetros variados

----

Taxa de crossover == ~0.8~ => **0.6**

Taxa de mutação == ~0.1~ => **0.3**

Tamanho da população == ~100~ => **80**

Geração máxima para parada == ~500~ => **1500**

Seguindo a lógica acima, rodamos o algoritmo com os parâmetros variados mais 50 vezes. 

In [ ]:
random.seed(SEED)

In [ ]:
generation_list = []
for i in range(50):
  chromosome, generation, fitness_values, maze, _, _ = run(maze_size=10, population_size=80, crossover_rate=0.6, mutation_rate= 0.3, max_generation=1500,n_generation_growth= 5)
  generation_list.append(generation)

print("Média de gerações para convergência do Algoritmo Genético: ", np.mean(np.array(generation_list)))

Aparentemente, a média de gerações para a convergência do Algoritmo Genético com os parâmetros variados tendeu a diminuir.

O gráfico a seguir é referente à ultima execução do algoritmo.



In [ ]:
show_graph(generation, fitness_values)

#### Utilizando outra função de fitness (Distância de Manhattan)

No exemplo a seguir utilizaremos a função fitness com Distância de Manhattan ao invés da Distância Euclideana.

In [ ]:
random.seed(SEED)

In [ ]:
generation_list = []
for i in range(50):
  chromosome, generation, fitness_values, maze, _, _ = run(maze_size=10, population_size=100, crossover_rate=0.8, mutation_rate= 0.1, max_generation=500,n_generation_growth= 5, fitness_function= "MANHATTAN")
  generation_list.append(generation)

print("Média de gerações para convergência do Algoritmo Genético: ", np.mean(np.array(generation_list)))

Ao alterar a função fitness para utilizar a Distância de Manhattan, notamos uma piora na convergência do algoritmo genético com os mesmos parâmetros base (artigo).

O gráfico a seguir é referente à ultima execução do algoritmo.

In [ ]:
show_graph(generation, fitness_values)


#### Efeito da variação dos parâmetros na velocidade de convergência da solução


O algoritmo genético com a variação dos parâmetros e função fitness de Distância Euclidiana convergiu mais rapidamente que o algoritmo com os parâmetros base.

Enquanto o AG com os parâmetros base (artigo) e Distância Euclidiana como função fitness encontrou a solução em uma média de **50.74** gerações, o AG com parâmetros variados sugeridos por todos os integrantes do grupo convergiu em uma média de **45.50** gerações.

Em uma variação da função fitness para utilizar a Distância de Manhattan, o AG convergiu em uma média de **65.5** gerações.

